In [ ]:
import os
import numpy as np
import pandas as pd
import cv2 as cv 
import matplotlib.pyplot as plt
import random

# OpenCV Laplacian w CLAHE mods
  
Laplacian and FFT seems to still be the sota method for blur and sharpness   quantification   
literature review on methods : https://www.researchgate.net/publication/234073157_Analysis_of_focus_measure_operators_in_shape-from-focus   

   
     
## Thoughts (edit): 
1.   Use CLAHE to reduce noise-amplification, reduce brightness, from wet areasm and enhance image feeatures.      
1b.  experiment with `tileGridSize` and `clipLimit`   
1c.  also experiment with contrast limiting? (prelim results don't look promising)   

In [ ]:
root = "..."
# take 10 images
file_names = os.listdir(root)[:10]
images = []
for name in file_names: images += [cv.imread(root + name)]
# images[0]

## Helper Functions: Image Display (edit)
use `preview(img, width = 200)` :. `preview(image)` to display images in smaller cell    
`preview` combines `cv2_imshow`, `.copy()`, and `.resize()`

In [ ]:
# -- HELPER FUNCTIONS: IMAGE DISPLAY

# shrink display size without altering original image
def shrink(img, width=None, height=None, inter=cv.INTER_AREA):
  # set width OR height pixels
  image = img.copy()
  dim = None
  (h, w) = image.shape[:2]
  
  if width and height: return image
  if height:
      r = height / float(h)
      dim = (int(w * r), height)
  else:
      r = width / float(w)
      dim = (width, int(h * r))
  return cv.resize(image, dim, interpolation=inter)

# SHORTCUT TO SHRINK + CV2_IMSHOW
def preview(image, width = 200):
  image = image.copy()
  return shrink(image, width = width)

## Preprocessing: Cropping, Reflection Mask (`ref_th` edit)

In [ ]:
#Image cropping
def crop_img(img):
  # Remove around 100 pixels from the start and end of the image in booth axes of the given (1080 x 1350) image
  return img[100:1080 -100,100:1350-100]

# remove wet reflections 
import cv2
def get_reflections_mask(img_RGB, ref_th=0.875, expand_kernel=7):
    # if img_gray.ndim == 3 and img_gray.shape[2] > 1:
    #     img_gray = cv2.cvtColor(img_gray, cv2.COLOR_BGR2GRAY)
    YUV = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2YUV)
    reflection_mask = (YUV[:,:,0] < ref_th * 255).astype('uint8')
    reflection_mask = cv2.erode(reflection_mask,
                                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (expand_kernel, expand_kernel))).astype('uint8')
    return reflection_mask.astype('bool')

def inpaint_mask(rgb, mask):
    inpainted = cv2.inpaint(rgb, mask.astype('uint8'), inpaintRadius=13, flags=cv2.INPAINT_TELEA)
    return inpainted
  

## CLAHE contrast limited & other tests (edit)
from documentation:   
"So in a small area, histogram would confine to a small region (unless there is noise). If noise is there, it will be amplified. To avoid this, contrast limiting is applied. If any histogram bin is above the specified contrast limit (by default 40 in OpenCV), those pixels are clipped and distributed uniformly to other bins before applying histogram equalization.
   

contrast limiting separates our images into two phases, enabling us to further tune white vs darkspaces. Preliminary results (from test_clahe) don't look too good; **future directions should test clip limit and tile size**

In [ ]:
# EVALUATE DIFF BTWN REG HIST EQ AND CLAHE
def test_equalizer(test_image, mode = 'compare'):
  n_img = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
  eq = False
  c = False
  
  print('original image:')
  preview(n_img)

  if mode == 'equalize' or mode == 'compare':
    n_eq = cv.equalizeHist(n_img)
    print('normal histogram equalization:')
    preview(n_eq)

  if mode == 'clahe' or mode == 'compare':
    n_clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    n_cl = n_clahe.apply(n_img)
    print('clahe: ')
    preview(n_cl)
  

  # before eq: yellow, after: green
  i = plt.hist(n_img.flat, alpha = 0.5, bins=100, range=(0, 255), color='orange')
  j = plt.hist(n_eq.flat, alpha = 0.5, bins=100, range=(0, 255), color='r')
  k = plt.hist(n_cl.flat, alpha = 0.5, bins=100, range=(0, 255), color='g') 

# test_equalizer(images[0])

In [ ]:
# -- PARAMETER TUNE
clip_limit = 2.         # 2.0
tile_size = (8, 8)    # (8,8)
# -- PARAMETER TUNE

In [ ]:
# TEST CLAHE uses from PARAMETER TUNING, visualizing hist, setting threshold values

def test_clahe(image, tune = False, hist = False, threshold = False):
  if tune: print('original bw image')
  preview(image)
  n_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # standard state clahe
  n_clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  n_cl = n_clahe.apply(n_img)

  if tune: print('standard state CLAHE')

  preview(n_cl)
  
  if tune:
    # -- PARAMETER TUNE
    clip_limit = 2.         # 2.0
    tile_size = (8, 8)    # (8,8)
    # -- PARAMETER TUNE
    print('test parameters: \n clip limit: ', clip_limit, '\n tile size: ', tile_size)
    n_clahe2 = cv.createCLAHE(clipLimit = clip_limit, 
                              tileGridSize = tile_size)

    n_cl2 = n_clahe2.apply(n_img)
    preview(n_cl2)

  if hist:
    # before eq: yellow, after: green
    i = plt.hist(n_img.flat, alpha = 0.5, bins=100, range=(0, 255), color='y')
    k = plt.hist(n_cl.flat, alpha = 0.5, bins=100, range=(0, 255), color='g') 

  if threshold:
    print('attempt low, high, auto thresholding')
    # lower thresh
    i, thresh_low = cv2.threshold(n_cl, 150, 100, cv2.THRESH_BINARY)
    preview(thresh_low)
    # higher
    j, thresh_high = cv2.threshold(n_cl, 150, 200, cv2.THRESH_BINARY_INV)
    preview(thresh_high)
    # find threshhold automatically
    k, thresh_auto = cv2.threshold(n_cl, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    preview(thresh_auto)

# test_clahe(images[7], tune = True, hist = True)

## Histogram Equalization   

In [ ]:
# Images before histogram equalization w/ histogram plots

def histogram_eq(prev_step, verbose = False):
  i = 0
  
  # Conduct histogram equalization
  equalized = []
  for image in prev_step:
    image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # or convert
    equ = cv.equalizeHist(image_bw)
    equalized.append(equ)
    res = np.hstack((image_bw,equ)) # stacking images side-by-side
    if verbose: preview(res)

  if verbose: 
    for image in prev_step:
      hist, bins = np.histogram(image[0].flatten(),256,[0,256])
      cdf = hist.cumsum()
      cdf_normalized = cdf * hist.max()/ cdf.max()

      print("Image" , i)
      preview(image)
      plt.plot(cdf_normalized, color = 'b')
      plt.hist(image.flatten(),256,[0,256], color = 'r')
      plt.xlim([0,256])
      plt.legend(('cdf','histogram'), loc = 'upper left')
      plt.show()
      i+=1
    
  return equalized 

# histogram_eq(images, verbose = False)

## CLAHE

In [ ]:
# Trying Contrast Limited Adaptive Histogram Equalization (CLAHE)- histogram 
# equalization in specific tile regions to reduced information loss

def clahe(no_ref_images, verbose = True, show_all = False):
  clahe_images = []
  stacked1 = []
  for img in no_ref_images:
    if verbose : preview(img)
    image = cv.cvtColor(img, cv2.COLOR_BGR2GRAY) # or convert 
    # SET PARAMETERS
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size) # createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(image)
    if verbose: preview(cl1)
    clahe_images.append(cl1)
    res = np.hstack((image, cl1)) #stacking images side-by-side
    stacked1.append(res)
    if verbose: print('--')

  if show_all:
    equalized = histogram_eq(no_ref_images)
    for i in range(len(stacked1)):
      res = np.hstack((stacked1[i], equalized[i]))
      cv2.putText(res, "Original", (20, 35),
        cv.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3);
      cv2.putText(res, "CLAHE", (1200, 35),
        cv.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3);
      cv2.putText(res, "equalized", (2350, 35),
        cv.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3);
      preview(res, width = 800)

  return clahe_images

clahe_images = clahe(images, show_all=False)